In [ ]:
! kaggle competitions download -c 'geolifeclef-2022-lifeclef-2022-fgvc9'

100% 57.6G/57.6G [06:50<00:00, 218MB/s]
100% 57.6G/57.6G [06:50<00:00, 151MB/s]


In [ ]:
! mkdir train
! unzip 'geolifeclef-2022-lifeclef-2022-fgvc9.zip' -d train

Streaming output truncated to the last 5000 lines.
  inflating: train/patches-us/20/82/21328220_landcover.tif  
 extracting: train/patches-us/20/82/21328220_near_ir.jpg  
  inflating: train/patches-us/20/82/21328220_rgb.jpg  
  inflating: train/patches-us/20/82/21368220_altitude.tif  
  inflating: train/patches-us/20/82/21368220_landcover.tif  
 extracting: train/patches-us/20/82/21368220_near_ir.jpg  
  inflating: train/patches-us/20/82/21368220_rgb.jpg  
  inflating: train/patches-us/20/82/21378220_altitude.tif  
  inflating: train/patches-us/20/82/21378220_landcover.tif  
  inflating: train/patches-us/20/82/21378220_near_ir.jpg  
  inflating: train/patches-us/20/82/21378220_rgb.jpg  
  inflating: train/patches-us/20/82/21398220_altitude.tif  
  inflating: train/patches-us/20/82/21398220_landcover.tif  
 extracting: train/patches-us/20/82/21398220_near_ir.jpg  
  inflating: train/patches-us/20/82/21398220_rgb.jpg  
  inflating: train/patches-us/20/82/21408220_altitude.tif  
  inflati

In [ ]:
import pandas as pd
df_obs_fr = pd.read_csv("train/observations/observations_fr_train.csv", sep=";", index_col="observation_id")
df_obs_us = pd.read_csv("train/observations/observations_us_train.csv", sep=";", index_col="observation_id")

df_obs = pd.concat((df_obs_fr, df_obs_us))

print("Number of observations for training: {}".format(len(df_obs)))

df_obs.head()

Number of observations for training: 856393


,latitude,longitude,species_id,subset
observation_id,,,,
10561949,45.705116,1.424622,241.0,train
10131188,45.146973,6.416794,101.0,train
10799362,46.783695,-2.072855,700.0,train
10392536,48.604866,-2.825003,1456.0,train
10335049,48.815567,-0.161431,157.0,train


In [ ]:
df_obs_fr_test = pd.read_csv("train/observations/observations_fr_test.csv", sep=";", index_col="observation_id")
df_obs_us_test = pd.read_csv("train/observations/observations_us_test.csv", sep=";", index_col="observation_id")

df_obs_test = pd.concat((df_obs_fr_test, df_obs_us_test))

print("Number of observations for testing: {}".format(len(df_obs_test)))

df_obs_test.head()

Number of observations for testing: 36421


,latitude,longitude
observation_id,,
10782781,43.601788,6.940195
10364138,46.241711,0.683586
10692017,45.181095,1.533459
10222322,46.938450,5.298678
10241950,45.017433,0.960736


In [ ]:
!rm -rf GLC
!git clone https://github.com/maximiliense/GLC

Cloning into 'GLC'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 383 (delta 119), reused 170 (delta 63), pack-reused 155
Receiving objects: 100% (383/383), 10.57 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [ ]:


df_suggested_landcover_alignment = pd.read_csv("train/metadata/landcover_suggested_alignment.csv", sep=";")
df_suggested_landcover_alignment.head()

,landcover_code,suggested_landcover_code,suggested_landcover_label
0,0,0,Missing Data
1,1,11,Cultivated Crops
2,2,11,Cultivated Crops
3,3,6,Broad-leaved Forest
4,4,7,Coniferous Forest


In [ ]:
from GLC.data_loading.common import load_patch

patch = load_patch(10171444, DATA_PATH)

print("Number of data sources: {}".format(len(patch)))
print("Arrays shape: {}".format([p.shape for p in patch]))
print("Data types: {}".format([p.dtype for p in patch]))

Number of data sources: 4
Arrays shape: [(256, 256, 3), (256, 256), (256, 256), (256, 256)]
Data types: [dtype('uint8'), dtype('uint8'), dtype('int16'), dtype('uint8')]


In [ ]:
landcover_mapping = df_suggested_landcover_alignment["suggested_landcover_code"].values
#patch = load_patch(10171444, DATA_PATH, landcover_mapping=landcover_mapping)

In [ ]:
obs_id_train = df_obs.index[df_obs["subset"] == "train"].values
obs_id_val = df_obs.index[df_obs["subset"] == "val"].values

y_train = df_obs.loc[obs_id_train]["species_id"].values
y_val = df_obs.loc[obs_id_val]["species_id"].values

n_val = len(obs_id_val)
print("Training set size: {} ({:.1%} of train observations)".format(len(y_train), len(y_train) / len(df_obs)))
print("Validation set size: {} ({:.1%} of train observations)".format(n_val, n_val / len(df_obs)))

Training set size: 836508 (97.7% of train observations)
Validation set size: 19884 (2.3% of train observations)


In [ ]:
import numpy as np
import sys
def load_train_data(factor):
    X_train = list() #np.array((np.shape(y_train), 256, 256, 3))
    for obs_id in obs_id_train:
        patch = load_patch(obs_id, "train", landcover_mapping=landcover_mapping)
        X_train.append(patch[0])

        percent_progress = len(X_train)/(len(obs_id_train)/factor) * 100
        sys.stdout.write('\r')
        # the exact output you're looking for:
        sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
        sys.stdout.flush()

        if len(X_train) >= (len(obs_id_train)/factor):
            break
    print()
    
    X_train = np.array(X_train)
    return X_train

def load_val_data(factor):
    X_val = list() #np.array((np.shape(y_train), 256, 256, 3))
    for obs_id in obs_id_val:
        patch = load_patch(obs_id, "train", landcover_mapping=landcover_mapping)
        X_val.append(patch[0])

        percent_progress = len(X_val)/(len(y_val)/factor) * 100
        sys.stdout.write('\r')
        # the exact output you're looking for:
        sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
        sys.stdout.flush()

        if len(X_val) >= (len(y_val)/factor):
            break

    print()

    X_val = np.array(X_val)

In [ ]:
factor = 1000
# train_ds = tf.data.Dataset.from_tensor_slices((load_train_data(factor), y_train[:obs_id_train//factor]))
# train_ds = train_ds.batch(64)

num_classes = len(set(df_obs['species_id']))
input_shape = (256, 256, 3)



In [ ]:
subset_size = 1000
obs_list = list()

# iterate over a subset of the labels
counter = 0
for y in np.unique(y_train)[:subset_size]:
    # for each label, retrieve all corresponding observation ids
    obs = df_obs.index[df_obs["species_id"] == y].values
    obs_list.append(obs)
    
# we now have a numpy array of all observation ids corresponding to this subset of labels
obs_id_train = np.concatenate(obs_list)

# obtain the labels in the right order 
y_train = df_obs.loc[obs_id_train]["species_id"].values

## **RESNET**

In [ ]:
import tensorflow as tf
def identity_block(x, filter):
  x_skip = x
  x = tf.keras.layers.Conv2D(filter[0], (3,3), padding = "same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  #print("A")
  x = tf.keras.layers.Conv2D(filter[1], (3,3), padding = "same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, x_skip])

  x = tf.keras.layers.Activation('relu')(x)

  return x






In [ ]:
def conv_net(x, filter):
  
  x_skip = x
  #print("B")
  x = tf.keras.layers.Conv2D(filter[0], (3,3), strides = 2, padding = "same")(x)
  #print("C")
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  #print("D")
  x = tf.keras.layers.Conv2D(filter[0], (3,3), padding = "same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  #print("E")
  x_skip = tf.keras.layers.Conv2D(filter[1], (1,1), strides = 2, padding = "same")(x_skip)
  x = tf.keras.layers.Add()([x, x_skip])
  #print("F")
  x = tf.keras.layers.Activation('relu')(x)

  return x



In [ ]:
def my_res(input_shape, num_classes):
  x_input = tf.keras.layers.Input(input_shape)
  x = tf.keras.layers.ZeroPadding2D(padding = (3,3))(x_input)

  #print("a")
  x = tf.keras.layers.Conv2D(64, 7 , strides = 2, padding = "same")(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides = (2,2), padding = "same")(x)

  #print(x.shape)
  #print('b')
  x = conv_net(x, (64, 64))
  #print('c')
  x = identity_block(x, (64, 64))

  #print('d')
  x = conv_net(x, (128, 128))
  x = identity_block(x, (128, 128))
  x = identity_block(x, (128, 128))
  #x = identity_block(x, (128, 128))

  x = conv_net(x, (256, 256))
  x = identity_block(x, (256, 256))
  x = identity_block(x, (256, 256))
  x = identity_block(x, (256, 256))
  x = identity_block(x, (256, 256))

  x = tf.keras.layers.AveragePooling2D(padding = "same")(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(num_classes, activation = 'softmax')(x)

  model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "resnet_from_scratch")

  return model



In [ ]:


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001 )
num_classes = len(set(df_obs['species_id']))
input_shape = (256, 256, 3)
model = my_res(input_shape, num_classes)
# Compile model
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[
                  tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(10, name="top-10-accuracy")
              ]
              )

In [ ]:
obs_id_train = (df_obs.index[df_obs["subset"] == "train"].values)
obs_id_val = (df_obs.index[df_obs["subset"] == "val"].values)

y_train = (df_obs.loc[obs_id_train]["species_id"].values)
y_val = (df_obs.loc[obs_id_val]["species_id"].values)

# load training dataset samples
# factor = 1 means load full training dataset
# factor = 100 means load 1/100 of the full dataset
factor = 1

X_train = list() #np.array((np.shape(y_train), 256, 256, 3))
for obs_id in obs_id_train:
    patch = load_patch(obs_id, "train", landcover_mapping=landcover_mapping)
    X_train.append(patch[0])
    
    percent_progress = len(X_train)/(len(y_train)/factor) * 100
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
    sys.stdout.flush()
        
    if len(X_train) >= (len(y_train)/factor):
        break
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train[:len(X_train)]))

train_ds = train_ds.batch(64)

# load validation dataset samples
factor = 1

X_val = list() #np.array((np.shape(y_train), 256, 256, 3))
for obs_id in obs_id_val:
    patch = load_patch(obs_id, "train", landcover_mapping=landcover_mapping)
    X_val.append(patch[0])
    
    percent_progress = len(X_val)/(len(y_val)/factor) * 100
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
    sys.stdout.flush()
    
    if len(X_val) >= (len(y_val)/factor):
        break


    
X_val = np.array(X_val)
print("sample array shape: ", np.shape(X_val))
print("label array shape: ", np.shape(y_val))

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val[:len(X_val)]))
val_ds = val_ds.batch(64)
    
X_train = np.array(X_train)
print("sample array shape: ", np.shape(X_train))
print("label array shape: ", np.shape(y_train))


[====================] 100%

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.001, patience=5, 
                                              verbose=0, mode='auto', baseline=None, restore_best_weights=True)
train_results = model.fit(
    train_ds,
    validation_data= val_ds,
    epochs=100, callbacks=[early_stop])